In [1]:
import csv
import random
from scipy import spatial
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from numpy import linspace
import simplekml
from pylab import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Implementing the prediction, simulation and visualization class

In [2]:
class PierrePredictor:
    # Gaussian Process Regression
    
    def __init__(self, data):
        # The dimension of the data
        self.n_dim = len(data)
        
    def K(self, X, Z, h):
        # The Covariance Matrix
        d = spatial.distance_matrix(X,Z)
        K = np.exp(-(d**2) / (2*h*h)) 
        return K
    
    def mean_f(self,h,sigma,X,Z,Y):
        # The predictive mean m(f)
        K_X_X = self.K(X,X,h)
        K_test_X = self.K(Z,X,h)
        Id = np.identity(X.shape[0])
        S=np.linalg.inv(K_X_X+sigma*Id)
        m=np.dot(K_test_X,np.dot(S,Y))
        return m

    def cov_f(self,h,sigma,X,Z):
        # The predictive cov(f)
        K_test = self.K(Z,Z,h)
        K_test_X = self.K(Z,X,h)
        K_X_test = self.K(X,Z,h)
        K_X_X = self.K(X,X,h)
        Id = np.identity(X.shape[0])
        S=np.linalg.inv(K_X_X+sigma*Id)
        cov_f = K_test-np.dot(K_test_X,np.dot(S,K_X_test))
        return cov_f
        
    def L(self,h,sigma,X,Z):
        # The Cholesky decomposition of the predictive cov(f)
        cov_f = self.cov_f(h,sigma,X,Z)
        L = np.linalg.cholesky(cov_f + 0.001*np.eye(cov_f.shape[0]))
        return L
    
    def f_sim(self,h,sigma,X,Z,Y):
        # The conditional stochastic simulations of f
        m = self.mean_f(h,sigma,X,Z,Y)
        L = self.L(h,sigma,X,Z)
        mean_f_u = np.zeros(Z.shape[0])
        Id = np.identity(Z.shape[0])
        u = np.random.multivariate_normal(mean_f_u,Id)
        u.shape=Z.shape[0],1
        fsim = m + np.dot(L,u)
        return fsim
    
    def visualization(self,fsim,n_squared_grid,bounding_box):
        
        # bounding_box = List of the latitude & longitude of the bounding box
        # We reshape fsim in a grid of n_squared_grid * n_squared_grid
        
        fsim_reshaped = np.transpose(np.asarray(np.array_split(fsim,n_squared_grid)))
        
        # Exportation of the picture in a .png file
        
        fig = plt.figure(2)

        cmap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap',
                                                   ['darkblue','blue','lightblue','green','lightgreen','yellow'
                                                    ,'orange','red','darkred']
                                                    ,256)

        img = plt.imshow(fsim_reshaped,interpolation='nearest',cmap = cmap,origin='lower')

        fig.savefig("Picture of the simulated rainfall.png",transparent=True,frameon=False,pad_inches=0,bbox_inches='tight')
        
        # Exportation of the picture in a .kml file, readable by Google Earth
        
        kml = simplekml.Kml()
        ground = kml.newgroundoverlay(name='GroundOverlay')
        ground.icon.href = 'Picture of the simulated rainfall.png'
        
        ground.latlonbox.north = bounding_box[1]
        ground.latlonbox.south = bounding_box[0]
        ground.latlonbox.east = bounding_box[2]
        ground.latlonbox.west = bounding_box[3]
        ground.latlonbox.rotation = 0

        ground.color = "aaffffff"
        kml.save("Picture of the simulated rainfall.kml")
            
            

# Importing the data set

In [3]:
data = np.genfromtxt('trn_data.csv', delimiter=',', skip_header=1)
X, Y = data[:,:-1], data[:,-1:]

In [4]:
predictor=PierrePredictor(data)

# Cross - validation

#### Creation of the k - fold.

In [5]:
predictor.n_dim

414

In [6]:
414 / 9

46

In [7]:
np.random.shuffle(data)
X_random, Y_random = data[:,:-1], data[:,-1:]
X_folds = np.array_split(X_random, 9)
Y_folds = np.array_split(Y_random, 9)

#### Definition of the RMSE function.

In [8]:
def rmse(predictions, targets):
            return np.sqrt(((predictions - targets) ** 2).mean())

## Cross - Validation on both h & sigma, using fsim

In [9]:
liste_rmse=[]
minimum_h=[]
minimum_sigma=[]
liste_h_sigma=[]

for i in range(9):
    X_train_cv = list(X_folds)
    X_test_cv  = X_train_cv.pop(i)
    X_train_cv = np.concatenate(X_train_cv)
    
    Y_train_cv = list(Y_folds)
    Y_test_cv  = Y_train_cv.pop(i)
    Y_train_cv = np.concatenate(Y_train_cv)
    
    for h in np.arange(0.05,3,0.05):
        for sigma in np.arange(0.05,1,0.05):
          
            mean = predictor.mean_f(h,sigma,X_train_cv,X_test_cv,Y_train_cv)
            
            liste_h_sigma.append([h,sigma])
            liste_rmse.append(rmse(mean,Y_test_cv))

    every_minimum=liste_h_sigma[liste_rmse.index(min(liste_rmse))]
    
    minimum_sigma.append(every_minimum[1])
    minimum_h.append(every_minimum[0])

    liste_rmse=[]
    liste_h_sigma=[]
    
best_h=sum(minimum_h)/float(len(minimum_h))
best_sigma = sum(minimum_sigma)/float(len(minimum_sigma))

print best_h, best_sigma

0.738888888889 0.255555555556


#### I fix h and sigma because their calculation take a very long time.

In [10]:
h_chosen_fixed = 0.738888888889
sigma_chosen_fixed = 0.255555555556

## Importation of the test file

In [11]:
test = np.genfromtxt('tst_locations.csv', delimiter=',', skip_header=1)
test.shape

(413L, 2L)

# Prediction with m(f) & exportation as a csv file

In [14]:
# Predictive mean #

mean = predictor.mean_f(h_chosen_fixed,sigma_chosen_fixed,X,test,Y)

# Exportation as a csv file for the kaggle competition #

t = open('Predicitve_mean_m(f).csv', 'w')
open_file_object = csv.writer(t)

open_file_object.writerow(['id','mm'])

for i in range(test.shape[0]):
    open_file_object.writerow([i+1,int(mean[i])])

t.close()

In [15]:
mean.shape

(413L, 1L)

# Simulation

In [17]:
grid = np.genfromtxt('grid.csv', delimiter=',')
grid.shape

(2500L, 2L)

In [18]:
fsim = predictor.f_sim(h_chosen_fixed,sigma_chosen_fixed,X,grid,Y)
fsim.shape

(2500L, 1L)

#### I save fsim in order to use it after, without having to recalculate everything

In [19]:
d=fsim.tolist()

fsim_csv = open('fsim_output.csv', 'w')
open_file_object = csv.writer(fsim_csv)

for i in range(grid.shape[0]):
    open_file_object.writerow(d[i])

fsim_csv.close()

# Visualization

In [20]:
# Importation of fsim #

fsim_2500 = np.genfromtxt('fsim_output.csv')
fsim_2500.shape

(2500L,)

In [21]:
predictor.visualization(fsim_2500,50,[38.5,39.3,-119.8,-120.8])